# 8. Reducing Training Bias

## 강의소개

- 8강에서는 Open-domain QA 문제를 풀 때 발생할 수 있는 bias에 대해 배워보겠습니다.
- Bias는 무엇인지, 어떤 상황에서 발생할 수 있는지 전반적인 내용들에 대해 알아본 후 ODQA를 풀 때 생길 수 있는 여러 측면의 bias에 대해 알아보겠습니다.
- 마지막으로 각각의 bias를 해결하기 위한 노력들에는 어떤 것이 있는지 알아보는 시간을 가지겠습니다.

<br>

## Further Reading

- [Latent Retrieval for Weakly Supervised Open Domain Question Answering](https://arxiv.org/abs/1906.00300)
- [Dense Passage Retrieval for Open-Domain Question Answering](https://arxiv.org/abs/2004.04906)

<br>

## Reference

- Seo et al., 2019. Real-Time Open-Domain Question Answering with Dense-Sparse Phrase Index
- Karpukhin et al., 2020. Dense Passage Retrieval for Open-Domain Question Answering
- Lee et al., 2019. Latent Retrieval for Weakly Supervised Open Domain Question Answering
- Rajpurkar et al., 2016. SQuAD: 100,000+ Questions for Machine Comprehension of Text
- [https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/Nikola_Tesla.html](https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/Nikola_Tesla.html)

<br>

## 8.1 Definition of Bias

### 8.1.1 Bias의 종류

#### 8.1.1.1 Bias in learning

- inductive bias
  - 학습할 때 과적합을 막거나 사전 지식을 주입하기 위해 특정 형태의 함수를 선호하는 것

<br>

#### 8.1.1.2 A Biased World

- historical bias
  - 현실 세계가 편향되어 있기 떄문에 모델에 원치 않는 속성이 학습되는 것
- co-occurrence bias
  - 성별과 직업 간 관계 등 표면적인 상관관계 때문에 원치 않는 속성이 학습되는 것

<br>

#### 8.1.1.3 Bias in Data Generation

- specification bias
  - 입력과 출력을 정의한 방식 때문에 생기는 편향
- sampling bias
  - 데이터를 샘플링한 방식 때문에 생기는 편향
- annotator bias
  - 어노테이터의 특성 때문에 생기는 편향

<br>

### 8.1.2 Gender Bias

- 대표적인 bias 예시
- 특정 성별과 행동을 연관시켜서 예측 오류가 발생
- ex) cooking이라는 개념을 봤을 때 모델이 여자가 cooking을 한다는 데이터를 많이 봤기 때문에 남자가 cooking을 하고 있는 경우에도 agent를 woman으로 예측하는 것을 볼 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1s70rmUyd5u7d2Co0vCAHvijDFCGrudAQ' width=800/>

- "어떤 사람이 의사다"라는 터키어 문장을 구글 번역기를 통해 영어로 번역하면 "그(He)는 의사다"라고 남성으로 번역하게 된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sC617ZaD-tTd3zYK6sH7VrmWXW5zHiwN' width=600/>

<br>

### 8.1.3 Sampling Bias

- ex) <리터러시 다이제스트> 여론조사 (1936년)
  - 표본 크기: 240만 명 (사상 최대)
  - 예측: 루즈벨트 43% 알프레드 랜던 57%
  - 실제: 루즈벨트 62% 알프레드 랜던 38%
  - 설문 대상: 잡지 정기구독자, 자동차 등록명부, 사교클럽 인명부 등
    - 중산층 이상으로 표본이 왜곡
  - 2년후 리터러시 다이제스트 파산

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sD6ePmzJNOeE5ragh8Ic7kAlnQ-lZF8f' width=600/>

<br>

## 8.2 Bias in Open-domain Question Answering

### 8.2.1 Retriever-Reader Pipeline

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sKVl-1M00dj-Z7FRbQublCcOYyX8IiSc' width=600/>

- We will focus on the bias in reader model

<br>

### 8.2.2 Training bias in reader model

- 만약 reader 모델이 한정된 데이터셋에서만 학습이 된다면 reader는 항상 정답이 문서 내에 포함된 데이터쌍(positive)만을 보게 됨
  - 특히 SQuAD와 같은 (Context, Query, Answer)이 모두 포함된 데이터는 positive가 완전히 고정되어 있음
- Inference 시 만약 데이터 내에 찾아볼 수 없었던 새로운 문서를 준다면?
  - Reader 모델은 문서에 대한 독해 능력이 매우 떨어질 것이고, 결과적으로 정답을 내지 못할 것임

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sMabcqSD8AfAp0Ys64DnV6cI6gMNcEWC' width=600/>

<br>

### 8.2.3 How to mitigate(완화시키다) training bias?

#### 8.2.3.1 Train negative examples

- 훈련할 때 잘못된 예시를 보여줘야 retriever가 negative한 내용들은 먼 곳에 배치할 수 있음
- Negative sample도 완전히 다른 negative와 비슷한 negative에 대한 차이 고려도 필요함
- 어떻게 **좋은 negative sample**을 만들 수 있을까? (5강 참고)
  - corpus 내에서 랜덤하게 뽑기
  - 좀 더 헷갈리는 negative 샘플들 뽑기
    - 높은 BM25 / Tf-IDF 매칭 스코어를 가지지만, 답을 포함하지 않는 샘플
    - 같은 문서에서 나온 다른 Passage/Question 선택하기

<br>

#### 8.2.3.2 Add no answer bias

- 입력 시퀀스의 길이가 N일 경우, 시퀀스의 길이 외 1개의 토큰이 있다고 생각하기
- 훈련 모델의 마지막 레이어 weight에 훈련 가능한 bias를 하나 더 추가
- softmax로 answer prediction을 최종적으로 수행할 때, start와 end의 확률이 해당 bias 위치에 있는 경우가 가장 확률이 높으면 이는 "대답할 수 없다"라고 취급

<br>

## 8.3 Annotation Bias from Datasets

### 8.3.1 What is a annotation bias?

- ODQA 학습 시 기존의 MRC 데이터셋 활용
- ODQA 세팅에는 적합하지 않은 bias가 데이터 제작(annotation) 단계에서 발생할 수 있음

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sPdM9Rn6m6CumLxPia_5_vN-3alIjySd' width=800/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sR-z8FmCaUyNSE-HfqWmFsr_L9GWXr_t' width=800/>

- 위와 같은 편향 때문에 원치 않는 경우가 발생할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1scVfObOiXaFTOrKwLTcG0CVwTqLDKOvk' width=800/>

- 질문을 하는 사람이 답을 알고 있음 (문제가 너무 쉬워짐)
- 질문과 evidence 문단 사이의 많은 단어가 겹치는 bias 발생 가능
- SQuAD
  - only 500+ wiki article
  - 학습 데이터의 분포 자체가 이미 bias 되어 있음

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sf6ETioZkrMgAYOYdgXhcFOQfJDgYlXy' width=600/>

- SQuAD의 경우 Question을 쓴 사람이 답을 알고 있을 뿐만 아니라 답이 어떤 paragraph에서 나오는 지도 알고 있다.

<br>

### 8.3.2 Effect of annotation bias

- 데이터셋별 성능 차이가 annotation bias로 인해 발생할 수 있음
  - BM25: sparse embedding
  - DPR: dense embedding

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sf9hHi-2HiDXyXNyROM4CA7N6SFbAz-g' width=800/>

- SQuAD에서는 BM25를 사용했을 때가 DPR을 사용했을 때보다 점수가 높다.
- 하지만 NQ를 포함한 다른 데이터셋에서는 이와 반대로 BM25를 사용했을 때가 DPR을 사용했을 때보다 점수가 낮다.
- 이러한 현상이 발생하는 이유는 SQuAD Annotation bias가 단어의 overlap을 유도하게 되고 이러한 overlap은 BM25에서 훨씬 유리한 방식이다.
- SQuAD의 이러한 문제는 BM25와 DPR 방식을 합침으로서 어느정도 해결할 수 있다.
  - 하지만 다른 데이터셋들에 대해서 BM25와 DPR을 합치는 방식은 SQuAD에 비해 성능 변화가 미비하다.

<br>

### 8.3.3 Dealing with annotation bias

- Annotation 단계에서 발생할 수 있는 bias를 인지하고, 이를 고려하여 데이터를 모아야 함
- ex) ODQA 세팅과 유사한 데이터  수집 방법: Natural Questions
  - Supporting evidence가 주어지지 않은, 실제 유저의 question들을 모아서 데이터셋을 구성
  - [https://ai.google.com/research/NaturalQuestions/visualization](https://ai.google.com/research/NaturalQuestions/visualization)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1nBNUKeWYHjIAlHgAJk74kcN_3qfiVEUO' width=800/>

<br>

### 8.3.4 Another bias in MRC dataset

- SQuAD는 Passage가 주어지고, 주어진 passage 내에서 질문과 답을 생성한다.
- 그렇기 때문에 ODQA에 applicable하지 않은 질문들이 존재한다.
  - ex) "미국의 대통령은 누구인가?"와 같은 질문은 ODQA에 적합하지 않은 질문이 될 수 있다. (언제의 대통령을 묻는 지에 대한 specific한 정보가 더 필요)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1sjjoBe-XmXpc8OsVMR0ssAxYxF05OnTs' width=800/>